<a href="https://colab.research.google.com/github/lkevin2018/cog-320-lecture-examples/blob/main/in_class_hf_resnet_colab_exercise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔍 Live Exercise: Using Google Colab + HuggingFace + ResNet

In this exercise, you'll:

1. Use **Google Colab** as your coding environment  
2. Load a **pretrained ResNet model** from **HuggingFace**  
3. Classify the main object in an image  
4. Try it on your **own image**  



## ✅ Step 1: Pre-reqs

1. Open this notebook in **Google Colab**.  
2. Turn on a GPU:
   - Click: `Runtime` → `Change runtime type`
   - Set **Hardware accelerator** to `GPU` or `TPU`
   - Click **Save**
3. Press Connect in the top right



In [ ]:
# ✅ Check that PyTorch is installed and whether a GPU is available
import torch

print("PyTorch version:", torch.__version__)
print("Is CUDA (GPU) available?", torch.cuda.is_available())

if torch.cuda.is_available():
    print("Using device:", torch.device("cuda"))
else:
    print("Using device: CPU")

## ✅ Step 2: Install HuggingFace libraries (if needed)

We’ll use:
- `transformers` → to load the ResNet model from HuggingFace
- `Pillow` (`PIL`) → to handle images


In [ ]:
# This may take a minute the first time.
# In Colab, the "!" runs a shell command.
!pip install -U transformers pillow

## ✅ Step 3: Import libraries

If this cell runs without errors, your environment + installs are working.


In [ ]:
from transformers import AutoImageProcessor, AutoModelForImageClassification
from PIL import Image
import requests
import torch

print("Imports successful ✅")

## ✅ Step 4: Load a test image from the internet

We’ll start with a **test image from a URL**.

### 👉 Your tasks:

1. Run the cell below as-is to test it.
2. Then **change the `image_url`** to any other image link (e.g. from Google Images, right-click → "Copy Image Address").
3. Run again and see if the model correctly guesses the object.

> Tip: Use simple images where one main object is centered (e.g., dog, cat, car, etc.).


In [ ]:
# You can replace this URL with another image URL
image_url = "https://images.pexels.com/photos/1108099/pexels-photo-1108099.jpeg"

response = requests.get(image_url, stream=True)
response.raise_for_status()  # make sure the download worked

image = Image.open(response.raw).convert("RGB")
display(image)

## ✅ Step 5: Load a ResNet model from HuggingFace

We willuse a pretrained **ResNet-50** model hosted on HuggingFace:

- Model ID: `microsoft/resnet-50`
- Task: Image classification
- Link: https://huggingface.co/microsoft/resnet-50

> Under the hood, this is:
> - Downloading model weights from HuggingFace
> - Loading them into memory
> - Preparing an image processor for preprocessing


In [ ]:
model_name = "microsoft/resnet-50"

# Load image processor and model from HuggingFace
processor = AutoImageProcessor.from_pretrained(model_name)
model = AutoModelForImageClassification.from_pretrained(model_name)

print("Loaded model:", model_name)

## ✅ Step 6: Classify the image

Now we’ll:
1. Preprocess the image with the HuggingFace image processor  
2. Run it through ResNet  
3. Show the **top-5 predicted labels**

### 👉 Your tasks:

- Run the cell once with the default image.  
- Then go back up, change `image_url` to a new image, re-run the **image cell**, then re-run this cell.  
- Observe how the labels change.


In [ ]:
# Make sure 'image' is defined (from previous cell)
inputs = processor(images=image, return_tensors="pt")

with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
probabilities = torch.nn.functional.softmax(logits, dim=-1)[0]

# Get top-5 predictions
top_k = 5
top_probs, top_indices = torch.topk(probabilities, top_k)

print("Top predictions:")
for prob, idx in zip(top_probs, top_indices):
    label = model.config.id2label[idx.item()]
    print(f"{label:<25} — {prob.item():.4f}")

## 🧪 Bonus Exercise: Upload **your own image**

Instead of using a URL, you can upload a file from your computer.

### 👉 Your tasks:

1. Run the cell below and use the upload dialog to pick an image (`.jpg`, `.png`, etc.).  
2. Make sure `image` is set to your uploaded photo.  
3. Re-run the **classification cell** (the one that shows top predictions).  
4. See if the model correctly identifies the main object.

> Try different images: pets, food, shoes, electronics, etc.


In [ ]:
from google.colab import files

uploaded = files.upload()

# Get the first uploaded file
for filename in uploaded.keys():
    print("You uploaded:", filename)
    image = Image.open(filename).convert("RGB")
    display(image)
    break

## 🎯 Wrap-Up & Extra Challenges

You’ve just:

- Used **Google Colab** as a coding + ML environment  
- Loaded a **pretrained ResNet model** from **HuggingFace**  
- Classified images from both a **URL** and **your own uploads**  

### ✅ Mini-Challenges (Try these on your own)

1. **Change the model**  
   - Replace `"microsoft/resnet-50"` with another image model from HuggingFace.
   - Example to try: `"microsoft/resnet-34"` or any other `image-classification` model on the Hub.

2. **Show only top-3 predictions**  
   - In the classification cell, change `top_k = 5` to `top_k = 3`.

3. **Print nicer output**  
   - Format the predictions as percentages (e.g., `85.3%` instead of `0.853`).

4. **Fail cases**  
   - Try confusing images and see where the model gets things wrong.  
   - Ask: *Why might it be wrong? What could improve it?*

---

You now have a working pipeline:
**Colab → HuggingFace model → ResNet → Image classification** ✅
